# Food Data Exploration for Data Visualization


## Notes

https://spoonacular.com/food-api/docs

Using Rapid API

## Installs

In [1]:
!pip install pandasql --quiet 

## Imports

In [2]:
# Imports
from google.colab import drive
import altair as alt
import pandas as pd
import numpy as np
import pandasql
import requests
import json
import ast
import os
import re

## Parameters

In [3]:
drive.mount('/content/drive')
#drive.flush_and_unmount()

Mounted at /content/drive


In [4]:
Key_Name = 'Robert'
with open('/content/drive/MyDrive/w209/Final/Keys/' + Key_Name, 'r') as f:
    Key = f.read()

In [5]:
Host = 'spoonacular-recipe-food-nutrition-v1.p.rapidapi.com'

# Get Recipes

##### Load Data

In [6]:
Ingredients = pd.DataFrame()
path = '/content/drive/MyDrive/w209/Final/Data/Ingredients/'
for file in os.listdir(path):
  data = pd.read_csv(path+file)
  data.drop_duplicates(subset=['id'], inplace=True)
  Ingredients = pd.concat([Ingredients, data])

Ingredients

,ingredients,id,title,cuisine
0,"[{'name': 'trimmed baby bok choy', 'image': 'b...",838297,Chicken Fried Rice,Chinese
1,"[{'name': 'chicken breast', 'image': 'chicken-...",608519,Kung Pao Chicken Lettuce Wraps,Chinese
2,"[{'name': 'cooked broccoli', 'image': 'broccol...",127730,Sweet and Spicy Chinese Five Spice Roasted Alm...,Chinese
3,"[{'name': 'broccoli florets', 'image': 'brocco...",1031836,Quinoa Fried Rice with Chicken and Vegetables,Chinese
4,"[{'name': 'brown sugar', 'image': 'light-brown...",112942,Spicy Chinese Chicken (Slow Cooker),Chinese
...,...,...,...,...
495,"[{'name': 'all purpose flour', 'image': 'flour...",328195,Chicken Jambalaya II,Cajun
496,"[{'name': 'cajun seasoning', 'image': 'chili-p...",542568,Cajun Shrimp and Quinoa Casserole,Cajun
497,"[{'name': 'bell pepper', 'image': 'bell-pepper...",316384,Vegetable Gumbo,Cajun
498,"[{'name': 'butter', 'image': 'butter-sliced.jp...",34805,Half-hour Chicken Gumbo,Cajun


In [7]:
ingredients = pd.DataFrame()
failed = []
for i in Ingredients.iterrows():
  data = i[1]['ingredients'].replace("'", "\"").replace("None", "\"None\"")
  try:
    data = json.loads(data)
    data = pd.json_normalize(data)
  except:
    failed.append(i[1]['id']) # Has issues with mixed ' and " as with contractions ([A-Za-z]+)[\'`]([A-Za-z]+) or possessives ([A-Za-z]+)[\'`](\s+)
    continue
  data['id'] = i[1]['id']
  data['title'] = i[1]['title']
  data['cuisine'] = i[1]['cuisine']
  data = data[['id', 'title', 'cuisine', 'name', 'amount.metric.value', 'amount.metric.unit', 'amount.us.value', 'amount.us.unit']]
  ingredients = pd.concat([ingredients, data])
ingredients

,id,title,cuisine,name,amount.metric.value,amount.metric.unit,amount.us.value,amount.us.unit
0,838297,Chicken Fried Rice,Chinese,trimmed baby bok choy,709.764,g,3.000000,cups
1,838297,Chicken Fried Rice,Chinese,black pepper,0.125,tsps,0.125000,tsps
2,838297,Chicken Fried Rice,Chinese,lean boneless chicken breast,453.592,g,1.000000,pound
3,838297,Chicken Fried Rice,Chinese,chicken broth,58.750,ml,0.250000,cup
4,838297,Chicken Fried Rice,Chinese,cornstarch,1.000,tsp,1.000000,tsp
...,...,...,...,...,...,...,...,...
2,244288,Cajun Cabbage Stew,Cajun,corn bread mix,226.796,g,8.000000,ounces
3,244288,Cajun Cabbage Stew,Cajun,egg,1.000,,1.000000,
4,244288,Cajun Cabbage Stew,Cajun,jalapeno,48.000,slice,48.000000,slice
5,244288,Cajun Cabbage Stew,Cajun,cooked frozen shredded meat,340.194,g,12.000000,ounces


In [8]:
path = '/content/drive/MyDrive/w209/Final/Data/Flattened/ingredients.csv'
ingredients.to_csv(path, mode='w')

In [9]:
Nutrition = pd.DataFrame()
path = '/content/drive/MyDrive/w209/Final/Data/Nutrition/'
for file in os.listdir(path):
  data = pd.read_csv(path+file)
  data.drop_duplicates(subset=['id'], inplace=True)
  Nutrition = pd.concat([Nutrition, data])

Nutrition

,badNutrition,goodNutrition,id,title,cuisine
0,"[{'title': 'Calories', 'amount': '556k', 'inde...","[{'title': 'Protein', 'amount': '12g', 'indent...",580502,Caribbean Easter Egg Dye,Caribbean
1,"[{'title': 'Calories', 'amount': '304k', 'inde...","[{'title': 'Protein', 'amount': '27g', 'indent...",1175851,Caribbean Rum Punch Cocktail,Caribbean
2,"[{'title': 'Calories', 'amount': '388k', 'inde...","[{'title': 'Protein', 'amount': '39g', 'indent...",1178513,McDonald’s Shamrock Shake – Copycat,Caribbean
3,"[{'title': 'Calories', 'amount': '304k', 'inde...","[{'title': 'Protein', 'amount': '30g', 'indent...",274003,Peppermint-Mocha Delight,Caribbean
4,"[{'title': 'Calories', 'amount': '414k', 'inde...","[{'title': 'Protein', 'amount': '28g', 'indent...",709884,Caribbean Truffle Pie,Caribbean
...,...,...,...,...,...
495,"[{'title': 'Calories', 'amount': '528k', 'inde...","[{'title': 'Protein', 'amount': '47g', 'indent...",328195,Chicken Jambalaya II,Cajun
496,"[{'title': 'Calories', 'amount': '481k', 'inde...","[{'title': 'Protein', 'amount': '40g', 'indent...",542568,Cajun Shrimp and Quinoa Casserole,Cajun
497,"[{'title': 'Calories', 'amount': '680k', 'inde...","[{'title': 'Protein', 'amount': '33g', 'indent...",316384,Vegetable Gumbo,Cajun
498,"[{'title': 'Calories', 'amount': '587k', 'inde...","[{'title': 'Protein', 'amount': '29g', 'indent...",34805,Half-hour Chicken Gumbo,Cajun


In [10]:
nutrition = pd.DataFrame()
failed = []
for n in Nutrition.iterrows():
  data = n[1]['badNutrition'].replace("'", "\"").replace("True", "\"True\"").replace("False", "\"False\"").replace("None", "\"None\"")
  try:
    data = json.loads(data)
    data = pd.json_normalize(data)
  except:
    failed.append(n[1]['id']) # Has issues with mixed ' and " as with contractions ([A-Za-z]+)[\'`]([A-Za-z]+) or possessives ([A-Za-z]+)[\'`](\s+)
    continue
  data['id'] = n[1]['id']
  data['name'] = n[1]['title']
  data['cuisine'] = n[1]['cuisine']
  data = data[['id', 'cuisine', 'name', 'title', 'amount', 'indented', 'percentOfDailyNeeds']]
  nutrition = pd.concat([nutrition, data])
  
nutrition

,id,cuisine,name,title,amount,indented,percentOfDailyNeeds
0,580502,Caribbean,Caribbean Easter Egg Dye,Calories,556k,False,27.84
1,580502,Caribbean,Caribbean Easter Egg Dye,Fat,29g,False,44.62
2,580502,Caribbean,Caribbean Easter Egg Dye,Saturated Fat,4g,True,25.37
3,580502,Caribbean,Caribbean Easter Egg Dye,Carbohydrates,65g,False,21.68
4,580502,Caribbean,Caribbean Easter Egg Dye,Sugar,5g,True,5.98
...,...,...,...,...,...,...,...
2,244288,Cajun,Cajun Cabbage Stew,Saturated Fat,0.4g,True,2.51
3,244288,Cajun,Cajun Cabbage Stew,Carbohydrates,12g,False,4.25
4,244288,Cajun,Cajun Cabbage Stew,Sugar,6g,True,7.56
5,244288,Cajun,Cajun Cabbage Stew,Cholesterol,8mg,False,2.75


In [11]:
path = '/content/drive/MyDrive/w209/Final/Data/Flattened/nutrition.csv'
nutrition.to_csv(path, mode='w')

In [12]:
taste = pd.DataFrame()
path = '/content/drive/MyDrive/w209/Final/Data/Taste/'
for file in os.listdir(path):
  data = pd.read_csv(path+file)
  data.drop_duplicates(subset=['id'], inplace=True)  
  data = data[['id', 'cuisine', 'title', 'sweetness', 'saltiness', 'sourness', 'bitterness', 'savoriness', 'fattiness', 'spiciness']]
  taste = pd.concat([taste, data])

taste

,id,cuisine,title,sweetness,saltiness,sourness,bitterness,savoriness,fattiness,spiciness
0,473895,European,Rigatoni Brussel Sprout Bake + Figs,100.00,16.06,34.38,14.20,10.75,65.81,1800000.0
1,204988,European,Parting Glass Belgian Dubbel (For Intermediate...,47.48,100.00,27.89,26.62,37.55,54.74,0.0
2,758105,European,Late Summer Minestrone with Butternut Squash a...,80.16,91.51,94.27,100.00,85.02,83.32,0.0
3,603272,European,Greek Salad Quinoa Bowl,31.41,15.69,36.00,25.57,2.50,100.00,10000.0
4,491863,European,Green Goddess Pasta Primavera,70.37,34.30,100.00,25.42,23.32,58.79,0.0
...,...,...,...,...,...,...,...,...,...,...
495,328195,Cajun,Chicken Jambalaya II,54.60,100.00,35.61,27.20,49.16,66.34,79450.0
496,542568,Cajun,Cajun Shrimp and Quinoa Casserole,22.83,100.00,13.39,16.57,70.14,64.23,0.0
497,316384,Cajun,Vegetable Gumbo,31.68,100.00,17.23,29.38,52.58,65.06,5950.0
498,34805,Cajun,Half-hour Chicken Gumbo,33.88,98.51,39.61,25.99,59.44,100.00,15100.0


In [13]:
path = '/content/drive/MyDrive/w209/Final/Data/Flattened/taste.csv'
taste.to_csv(path, mode='w')

In [14]:
# From here we can shape the flattened data to flow directly into data visuals
pandasql.sqldf("""
    SELECT *
      FROM ingredients
      LIMIT 10
""")

,id,title,cuisine,name,amount.metric.value,amount.metric.unit,amount.us.value,amount.us.unit
0,838297,Chicken Fried Rice,Chinese,trimmed baby bok choy,709.764,g,3.000,cups
1,838297,Chicken Fried Rice,Chinese,black pepper,0.125,tsps,0.125,tsps
2,838297,Chicken Fried Rice,Chinese,lean boneless chicken breast,453.592,g,1.000,pound
3,838297,Chicken Fried Rice,Chinese,chicken broth,58.750,ml,0.250,cup
4,838297,Chicken Fried Rice,Chinese,cornstarch,1.000,tsp,1.000,tsp
5,838297,Chicken Fried Rice,Chinese,fresh ginger,2.000,tsps,2.000,tsps
6,838297,Chicken Fried Rice,Chinese,grapeseed oil,2.000,Tbsps,2.000,Tbsps
7,838297,Chicken Fried Rice,Chinese,fresh mushrooms,8.000,,8.000,
8,838297,Chicken Fried Rice,Chinese,onion,1.000,,1.000,
9,838297,Chicken Fried Rice,Chinese,oyster sauce,2.000,Tbsps,2.000,Tbsps
